In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

Compare values from the new with the existing logger - Ruapehu Crater Lake
--

Data from the new logger is on local disk.

Data from the existing logger is retrieved from FITS

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

**Data files**

In [ ]:
newfile = 'Table1.csv'
dtstart = '2018-05-04T04:00:00'

**Read new data**

In [ ]:
df = pd.read_csv(newfile, parse_dates=True, index_col = 'Datetime')
#Use only data after logger was installed
new = df[df.index>=dtstart]

In [ ]:
new.tail()

**Get recent existing data - temperature**

In [ ]:
#existing
url = 'https://fits.geonet.org.nz/observation?siteID=RU001&typeID=t&days=100&methodID=logic'
texist = pd.read_csv(url, parse_dates=['date-time'], index_col=['date-time'], usecols = ['date-time', ' t (C)'])
texist.rename(columns={' t (C)':'temp'}, inplace=True)

**Get recent existing data - depth**

In [ ]:
#existing
url = 'https://fits.geonet.org.nz/observation?siteID=RU001A&typeID=z&days=100'
dexist = pd.read_csv(url, parse_dates=['date-time'], index_col=['date-time'], usecols = ['date-time', ' z (m)'])
dexist.rename(columns={' z (m)':'depth'}, inplace=True)

**Set date-time range**

In [ ]:
texist1 = texist[texist.index>=dtstart]
dexist1 = dexist[dexist.index>=dtstart]

**Temperature**

In [ ]:
#difference between new and existing
diff = new['Temp_PRT_degC'] - texist1['temp']

#plot
tmp = new['Temp_PRT_degC'].plot(figsize=(20,7), title='Temperature', label='new data', fontsize=12)
texist1['temp'].plot(ax=tmp, marker='o', label='existing data')
offset = pd.DataFrame()
offset['temp'] = texist1['temp'] + diff.mean()
offset['temp'].plot(ax=tmp, linestyle='--', label='existing, offset corrected')
tmp.title.set_size(20)
tmp.grid()
tmp.set_ylabel("Ruapehu Crater Lake - Temperature (deg C)")
tmp.set_ylim(ymin=18, ymax=28)
tmp.legend(loc='best')

In [ ]:
fig = tmp.get_figure()
fig.savefig('temperature_compare.png', dpi=100)

**Temperature differences as a histogram**

In [ ]:
bins = np.arange(diff.min(), diff.max(), 0.1)
h = diff.hist(bins=bins)

**Depth**

In [ ]:
#difference between new and existing
diff = new['Depth_USGS_OTT_meters'] - dexist1['depth']

#plot
dep = new['Depth_USGS_OTT_meters'].plot(figsize=(20,7), title='Ruapehu Crater Lake - Water Depth Above Sensor',fontsize=12, label='new data')
offset = pd.DataFrame()
offset['depth'] = dexist1['depth'] + diff.mean()
offset['depth'].plot(ax=dep, marker='o', label='existing, offset corrected')
dep.title.set_size(20)
dep.grid()
dep.set_ylabel("Depth (m)")
dep.legend(loc='best')

In [ ]:
fig = dep.get_figure()
fig.savefig('depth_compare.png', dpi=100)

**Depth difference**

In [ ]:
dd = diff.plot(figsize=(20,7), title='Ruapehu Crater Lake - Depth Difference Between Sensors',fontsize=12, marker='o')
dd.set_ylabel('New Depth - Existing Depth (m)')

In [ ]:
diff.describe()